In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import wcs
import sys

from astropy.utils.data import get_pkg_data_filename

In [2]:
def load_wcs_from_file(filename):
    # Load the FITS hdulist using astropy.io.fits
    hdulist = fits.open(filename)

    # Parse the WCS keywords in the primary HDU
    w = wcs.WCS(hdulist[0].header)

    # Print out the "name" of the WCS, as defined in the FITS header
    print(w.wcs.name)

    # Print out all of the settings that were parsed from the header
    w.wcs.print_contents()

    # Three pixel coordinates of interest.
    # Note we've silently assumed an NAXIS=2 image here.
    # The pixel coordinates are pairs of [X, Y].
    # The "origin" argument indicates whether the input coordinates
    # are 0-based (as in Numpy arrays) or
    # 1-based (as in the FITS convention, for example coordinates
    # coming from DS9).
    pixcrd = np.array([[0, 0], [24, 38], [45, 98]], dtype=np.float64)

    # Convert pixel coordinates to world coordinates
    # The second argument is "origin" -- in this case we're declaring we
    # have 0-based (Numpy-like) coordinates.
    world = w.wcs_pix2world(pixcrd, 0)
    print(world)

    # Convert the same coordinates back to pixel coordinates.
    pixcrd2 = w.wcs_world2pix(world, 0)
    print(pixcrd2)

    # These should be the same as the original pixel coordinates, modulo
    # some floating-point error.
    assert np.max(np.abs(pixcrd - pixcrd2)) < 1e-6

    # The example below illustrates the use of "origin" to convert between
    # 0- and 1- based coordinates when executing the forward and backward
    # WCS transform.
    x = 0
    y = 0
    origin = 0
    assert (w.wcs_pix2world(x, y, origin) ==
            w.wcs_pix2world(x + 1, y + 1, origin + 1))

In [164]:
#load_wcs_from_file('../Data_CometHolmes/holmes-0004.wcs')
import cairo

a =!/Users/anahoban/software/astrometry/bin/wcsinfo ../Data_CometHolmes/holmes-0010.wcs


In [158]:
# Data
all_files = !ls ../Data_CometHolmes
jpeg_files = all_files[0::2] #to get only the jpeg files
#getting the image numbers

image_number = []
for i in jpeg_files:
    image_number.append(i[7:11])

In [215]:
#getting the exif
import PIL.Image
img = []
exif_data = []
n=0
for i in range(len(jpeg_files)):
    img.append(PIL.Image.open('../Data_CometHolmes/' + jpeg_files[i]))
    exif_data.append(img[i].getexif())

time_date = []
# associate picture number with time 
for i in range(1299):
    time_date.append([image_number[i], exif_data[i].get(306)])

#get only the ones with values
time_date_final = [row for row in time_date if row[1] != None ]

#some entries have the form: '2007:10:30 14:29:03\x00', so let's remove the weird ending
for i in range(len(time_date_final)):
    if len(time_date_final[i][1]) == 20:
        time_date_final[i][1] = time_date_final[i][1][:-1]

#discrepency between paper and code: we have 11 more timestamped images !!!!

[['0001', '2007:11:01 21:44:30'], ['0003', '2007:11:06 19:02:26'], ['0004', '2007:11:08 00:37:30'], ['0010', '2007:11:06 23:49:01'], ['0011', '2007:11:06 23:49:01'], ['0042', '2007:11:04 17:04:49'], ['0064', '2007:11:05 18:16:11'], ['0074', '2007:11:06 17:30:57'], ['0078', '2007:10:24 23:56:47'], ['0106', '2007:10:31 10:32:19'], ['0127', '2007:10:28 10:03:53'], ['0139', '2007:11:20 13:10:51'], ['0215', '2007:12:18 02:45:32'], ['0219', '2007:11:02 19:14:37'], ['0270', '2007:12:08 17:36:17'], ['0274', '2007:11:24 19:57:20'], ['0276', '2007:10:28 16:32:09'], ['0279', '2007:11:22 17:50:30'], ['0280', '2007:11:22 16:04:39'], ['0281', '2007:11:24 19:56:54'], ['0282', '2007:11:22 10:39:20'], ['0283', '2007:11:24 22:15:59'], ['0284', '2007:11:07 09:36:44'], ['0285', '2007:11:24 19:56:18'], ['0286', '2007:11:17 09:55:03'], ['0302', '2007:10:30 14:29:03'], ['0303', '2007:10:30 14:28:47'], ['0307', '2007:11:02 11:43:37'], ['0308', '2007:11:02 11:43:51'], ['0314', '2007:11:10 08:47:06'], ['0316', 

In [165]:
time_date_final

['2007:11:01 21:44:30',
 '2007:11:06 19:02:26',
 '2007:11:08 00:37:30',
 '2007:11:06 23:49:01',
 '2007:11:06 23:49:01',
 '2007:11:04 17:04:49',
 '2007:11:05 18:16:11',
 '2007:11:06 17:30:57',
 '2007:10:24 23:56:47',
 '2007:10:31 10:32:19',
 '2007:10:28 10:03:53',
 '2007:11:20 13:10:51',
 '2007:12:18 02:45:32',
 '2007:11:02 19:14:37',
 '2007:12:08 17:36:17',
 '2007:11:24 19:57:20',
 '2007:10:28 16:32:09',
 '2007:11:22 17:50:30',
 '2007:11:22 16:04:39',
 '2007:11:24 19:56:54',
 '2007:11:22 10:39:20',
 '2007:11:24 22:15:59',
 '2007:11:07 09:36:44',
 '2007:11:24 19:56:18',
 '2007:11:17 09:55:03',
 '2007:10:30 14:29:03',
 '2007:10:30 14:28:47',
 '2007:11:02 11:43:37',
 '2007:11:02 11:43:51',
 '2007:11:10 08:47:06',
 '2007:11:14 22:39:46',
 '2007:11:01 09:38:31',
 '2007:11:01 09:50:50',
 '2007:10:29 19:26:38',
 '2007:11:07 22:13:48',
 '2007:11:04 12:14:40',
 '2007:11:15 20:21:18',
 '2007:10:30 19:50:30',
 '2007:10:30 19:50:35',
 '2007:10:30 19:50:44',
 '2007:11:01 01:39:03',
 '2008:01:16 21: